In [33]:
import pandas as pd
import numpy as np
import requests
import cv2 
from pytesseract import pytesseract
from PIL import Image
import datetime
from datetime import datetime
from dateutil import parser
import copy

In [2]:
pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [3]:
df = pd.read_json('subspace.json')
df

,expiry_image,expiring_at,expiring_at_from_model,created_at,whatsub_plan
0,https://cdn.subspace.money/w_subscriptions/Xk5...,2023-05-13,2023-05-13,2022-05-15 04:27:17.955815+00:00,"{'duration': 3, 'duration_type': 'months'}"
1,https://cdn.subspace.money/w_subscriptions/PKV...,2023-03-29,None,2022-06-10 11:28:46.053421+00:00,"{'duration': 1, 'duration_type': 'years'}"
2,https://cdn.subspace.money/w_subscriptions/80E...,2022-12-27,2022-12-27,2022-07-21 16:30:53.316893+00:00,"{'duration': 1, 'duration_type': 'months'}"
3,https://cdn.subspace.money/w_subscriptions/JIX...,2022-10-22,2022-08-22,2022-07-22 15:02:53.685566+00:00,"{'duration': 1, 'duration_type': 'months'}"
4,https://cdn.subspace.money/w_subscriptions/p4r...,2022-07-12,None,2022-06-12 05:16:00.847964+00:00,"{'duration': 1, 'duration_type': 'months'}"
...,...,...,...,...,...
6723,https://cdn.subspace.money/w_subscriptions/OYl...,2023-10-07,2023-10-07,2022-10-07 16:03:36.596915+00:00,"{'duration': 12, 'duration_type': 'months'}"
6724,https://cdn.subspace.money/w_subscriptions/5av...,2022-11-07,None,2022-10-07 16:06:06.037587+00:00,"{'duration': 1, 'duration_type': 'months'}"
6725,https://cdn.subspace.money/w_subscriptions/X4e...,2022-12-31,None,2022-10-07 17:25:59.667282+00:00,"{'duration': 1, 'duration_type': 'months'}"
6726,https://cdn.subspace.money/w_subscriptions/qmT...,2022-11-07,None,2022-06-14 08:22:25.590519+00:00,"{'duration': 1, 'duration_type': 'months'}"


Date Processing

In [4]:
# df['expiring_at']= df['expiring_at'].dt.to_pydatetime()

In [5]:
df.iloc[1,1]

Timestamp('2023-03-29 00:00:00')

Image Processing

In [6]:
def img_proc():
    image = cv2.imread('test.jpeg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Morph open to remove noise and invert image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    invert = 255 - opening
    cv2.imwrite('converted.jpg',invert)
    return invert

Text Extraction

In [7]:
def txt_exr(invert):
    data = pytesseract.image_to_string(invert, lang='eng', config='--psm 6')
    text=data.lower().split('\n')
    return text

Date Extraction

In [37]:
def date_exr(text):
    dates=[]
    for w in text:
        try:
            res = parser.parse(w, fuzzy=True)
            if res.time()==datetime.min.time():     #Only Extracts pure dates excluding things such as times on screenshot
                dates.append((res.date(),w))
        except:
            continue
    return(dates)

Accessing All Images

In [42]:
phrase=[]
for i in df.iloc[:1,0]:
    c=0
    r = requests.get(i)
    print(i)
    with open('test.jpeg', 'wb') as out:
        out.write(r.content)
    invert=img_proc()   #Image Processing
    text=txt_exr(invert)    #Text Processing
    date=date_exr(text)     #Date Extractoin
    edate=df.iloc[c,1].to_pydatetime().date()
    print(edate,date)
    dmin=abs((date[0][0]-edate).days)
    k=0
    
    for j in range(len(date)):
        if abs((date[j][0]-edate).days)<dmin:
            dmin = abs((date[j][0]-edate).days) 
            k=j
    phrase.append((date[k][0],date[k][1],edate))
            
    im = Image.open("test.jpeg")
im.show()

https://cdn.subspace.money/w_subscriptions/Xk5bMvR8J7I_H6ywBPgSe.jpeg
2023-05-13 [(datetime.date(2020, 1, 21), "first became member on 21 jan '20"), (datetime.date(2022, 10, 30), 'free deliveries on all up ta 30° extra off on'), (datetime.date(2023, 5, 13), "expires on 13 may '23"), (datetime.date(2022, 10, 15), 'routes bepress at sat opogrtsa at'), (datetime.date(2022, 10, 4), '; a oe 4 sh. ae'), (datetime.date(1131, 10, 13), 'biergarten hakuna matata 1131 bar +'), (datetime.date(2022, 10, 3), 'ts & . : . 3'), (datetime.date(2022, 10, 4), 'o . 4 ‘ ‘ es . . feast')]


In [40]:
phrase

[(datetime.date(2023, 5, 13), "expires on 13 may '23"),
 (datetime.date(2022, 10, 2), '{2 (feu account')]

In [21]:
phrase

["expires on 13 may '23",
 '{2 (feu account',
 'expires on december 27, 2022',
 'subscription renewal date: mon 22 aug 2022']

In [27]:
df.iloc[1,0]

'https://cdn.subspace.money/w_subscriptions/PKVCqar0VD_Bd5ZeH5u3b.jpeg'

In [ ]:
# res<datetime.datetime.now()

NameError: name 'res' is not defined

Converting Date from DataFrame into Usable Format

In [ ]:
edate=df.iloc[3,1].to_pydatetime()
edate

datetime.datetime(2022, 10, 22, 0, 0)

In [ ]:
df.iloc[0,0]

'https://cdn.subspace.money/w_subscriptions/Xk5bMvR8J7I_H6ywBPgSe.jpeg'